In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [18]:
import pandas as pd
import numpy as np
from quantfreedom.enums import *
from quantfreedom.simulate import get_order_settings
from quantfreedom.base import backtest_df_only, backtest_sim_6, plot_one_result
from quantfreedom.helper_funcs import create_os_cart_product_nb
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.utils.helpers import pretty



price_data = pd.read_hdf("../../tests/data/prices.hd5")
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
price_data['BTCUSDT']

candle_info,open,high,low,close
open_time,,,,
2020-10-01 00:00:00,"10,759.00","10,817.50","10,759.00","10,790.00"
2020-10-01 00:30:00,"10,790.00","10,790.50","10,775.00","10,777.00"
2020-10-01 01:00:00,"10,777.00","10,793.00","10,777.00","10,793.00"
2020-10-01 01:30:00,"10,793.00","10,836.00","10,792.50","10,822.00"
2020-10-01 02:00:00,"10,822.00","10,841.50","10,808.00","10,808.50"
...,...,...,...,...
2021-04-14 21:30:00,"62,703.50","62,812.50","62,615.50","62,795.00"
2021-04-14 22:00:00,"62,795.00","62,825.00","62,591.00","62,680.00"
2021-04-14 22:30:00,"62,680.00","62,993.00","62,680.00","62,849.50"


In [4]:
rsi_ind = from_talib(
    func_name='rsi',
    nickname='rsi',
    price_data=price_data,
    input_names = ['close'],
    parameters = {'timeperiod': [15, 20]}
)
entries = rsi_ind.is_below(
    user_args=[40, 50],
    candle_ohlc="close",
)

In [5]:
account_state = AccountState()
backtest_settings = BacktestSettings(
    divide_records_array_size_by=2,
    gains_pct_filter=10.0,
    total_trade_filter=10,
    upside_filter=0.0,
)
exchange_settings = ExchangeSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01, .02]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([3]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
)

In [6]:
os_cart_arrays = create_os_cart_product_nb(order_settings_arrays=order_settings_arrays)

In [7]:
list(price_data.columns.levels[0])

['BTCUSDT', 'EOSUSD', 'ETHUSD', 'LTCUSDT', 'XRPUSD']

In [8]:
strat_res_df = backtest_df_only(
    account_state=account_state,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data['EOSUSD'],
    entries=entries['EOSUSD'],
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 4
Total combinations of settings to test: 16

Total candles: 9,408
Total candles to test: 150,528


In [9]:
strat_res_df

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,2,2,67.00,17.04,89.55,0.94,170.36,"1,170.36"
1,2,3,67.00,17.04,89.55,0.94,170.36,"1,170.36"
2,2,0,67.00,13.72,89.55,0.94,137.22,"1,137.21"
3,2,1,67.00,13.72,89.55,0.94,137.22,"1,137.21"
4,0,2,67.00,12.77,88.06,0.86,127.74,"1,127.74"
5,0,3,67.00,12.77,88.06,0.86,127.74,"1,127.74"
6,0,0,67.00,11.22,88.06,0.85,112.24,"1,112.24"
7,0,1,67.00,11.22,88.06,0.85,112.24,"1,112.24"


In [19]:
pretty(get_order_settings(2,os_cart_arrays))

OrderSettings(
    increase_position_type = 4.0,
    leverage_type = 2.0,
    max_equity_risk_pct = 0.03,
    order_type = 1.0,
    risk_account_pct_size = 0.01,
    risk_reward = 5.0,
    sl_based_on_add_pct = 0.0001,
    sl_based_on_lookback = 50.0,
    sl_candle_body_type = 3.0,
    sl_to_be_based_on_candle_body_type = 0.0,
    sl_to_be_when_pct_from_candle_body = 0.0,
    sl_to_be_zero_or_entry_type = 0.0,
    static_leverage = 0.0,
    stop_loss_type = 1.0,
    take_profit_type = 1.0,
    tp_fee_type = 1.0,
    trail_sl_based_on_candle_body_type = 2.0,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct_from_candle_body = 0.03,
)


In [20]:
order_records_df = backtest_sim_6(
    account_state=account_state,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=exchange_settings,
    price_data=price_data["EOSUSD"],
    entries=entries["EOSUSD"],
    strat_indexes=[0, 2, 4],
    strat_res_df=strat_res_df,
)

In [21]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,equity,available_balance,cash_borrowed,cash_used,average_entry,fees_paid,leverage,...,possible_loss,entry_size,entry_price,exit_price,position_size,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,2,2,33,NaN,989.49,37.85,10.51,2.52,NaN,4.70,...,10.00,48.36,2.52,NaN,48.36,NaN,20.57,2.00,103.46,5.12
1,2,2,270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.76,NaN,NaN
2,2,2,271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.84,NaN,NaN
3,2,2,272,"1,006.09","1,006.09",NaN,NaN,NaN,0.06,NaN,...,NaN,NaN,NaN,2.84,NaN,6.09,NaN,NaN,NaN,NaN
4,2,2,351,NaN,995.52,36.04,10.57,2.55,NaN,4.50,...,10.06,46.61,2.55,NaN,46.61,NaN,21.48,2.00,107.98,5.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0,2,9329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.74,NaN,NaN
764,0,2,9330,"1,126.62","1,126.62",NaN,NaN,NaN,0.15,NaN,...,NaN,NaN,NaN,6.74,NaN,2.68,NaN,NaN,NaN,NaN
765,0,2,9387,NaN,"1,114.79",47.13,11.83,7.41,NaN,5.10,...,11.27,58.96,7.41,NaN,58.96,NaN,19.01,6.00,95.63,14.49
766,0,2,9392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.56,NaN,NaN


In [13]:
plot_one_result(
    strat_res_df=strat_res_df,
    strat_result_index=0,
    price_data=price_data["EOSUSD"],
    order_records_df=order_records_df,
)